In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

model_path = "clasificador_analisis/clasificador/clasificador_tono/basico/comparativa/ep_4/modelo_final"

tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)


In [ ]:
def predict_sentiment_with_neutral(text, model, tokenizer, alpha=1.2, neutral_threshold=0.5):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        logits = model(**inputs).logits
    probs = torch.softmax(logits, dim=1).squeeze().tolist()
    neutral_score = 1 - abs(probs[0] - 0.5) ** alpha - abs(probs[2] - 0.5) ** alpha
    pred_label = 1 if neutral_score > (1 - neutral_threshold) else int(torch.argmax(torch.tensor(probs)))
    return pred_label, probs

In [ ]:
df = pd.read_excel("clasificador_analisis/clasificador/clasificador_tono/basico/mejor_test.xlsx")
df = df.rename(columns={"text": "text", "label": "label_manual_num"})


In [8]:

preds_and_scores = df["text"].apply(lambda x: predict_sentiment_with_neutral(x, model, tokenizer))
df["label_predicted"] = preds_and_scores.apply(lambda x: x[0])
df["scores"] = preds_and_scores.apply(lambda x: x[1])
df[["score_negativo", "score_neutro", "score_positivo"]] = pd.DataFrame(df["scores"].tolist(), index=df.index)

# Métricas
print("\n📊 MATRIZ DE CONFUSIÓN:")
print(confusion_matrix(df["label_manual_num"], df["label_predicted"]))

print("\n📈 CLASIFICATION REPORT:")
print(classification_report(
    df["label_manual_num"],
    df["label_predicted"],
    target_names=["Negativo", "Neutro", "Positivo"],
    digits=3
))



📊 MATRIZ DE CONFUSIÓN:
[[192  27   8]
 [ 16  74  11]
 [  7  16 164]]

📈 CLASIFICATION REPORT:
              precision    recall  f1-score   support

    Negativo      0.893     0.846     0.869       227
      Neutro      0.632     0.733     0.679       101
    Positivo      0.896     0.877     0.886       187

    accuracy                          0.835       515
   macro avg      0.807     0.818     0.811       515
weighted avg      0.843     0.835     0.838       515

